In [ ]:
import numpy as np
import math
import cmath
# import scipy.integrate as integrate

Some constants.

In [ ]:
pi2 = 2.*math.pi
i2pi = 2.j*math.pi

M = 100 # Trapezoid rule points.
N = 64  # Truncation level.

dt = pi2/M

eit = np.exp(np.arange(0, M, dtype=float)*1j*dt)

Domain things, canonical test case.

    circle(-0.2517+0.3129i, 0.2377), ...
    circle(0.2307-0.4667i, 0.1557));

In [ ]:
dv = np.array([-0.2517+0.3129j, 0.2307-0.4667j])
qv = np.array([0.2377, 0.1557])
m = dv.size

dv0 = lambda j: dv[j-1] if j > 0 else 0.+0.j
qv0 = lambda j: qv[j-1] if j > 0 else 1.

Test function.

In [ ]:
def polesInHoles(z):
    '''Put singularity in each hole.'''
    
    w = np.zeros(z.shape, z.dtype)
    for d in dv:
        w += 1/(z - d)
        
    return w

Read ARMA.

In [ ]:
def readMatrixFromFile(fname):
    '''Read matrix from file fname.'''
    
    f = open(fname, 'r')

    line = f.readline()
    if not line.startswith('ARMA_MAT_TXT'):
        raise Excetption('Not ARMA!')

    line = f.readline()
    rows, cols = line.strip('\n').split(sep=' ')
    (rows, cols) = (int(rows), int(cols))
    L = np.zeros((rows, cols), dtype=np.complex64)
    L.shape

    i = 0
    for line in f:
        x = line.strip(' \n').split(' ')
        L[i,:] = [complex(float(a[0]), float(a[1])) for a in [s.strip('()').split(',') for s in x]]
        i += 1

    f.close()
    
    return L

refRHS = readMatrixFromFile('../refRHS.dat')

Points on the boundary and imaginary part of function there.

In [ ]:
zb = np.empty((eit.size, m+1), dtype=np.complex_)
for j in range(0, m+1):
    zb[:,j] = dv0(j) + qv0(j)*eit
    
rz = polesInHoles(zb).imag

Matrix to do integration.

In [ ]:
L = np.empty((N+m*(N+1), (m+1)*M), dtype=np.complex_)

First row shares common bits with everything else, then $p=0$ is made of powers of $z_j$.

In [ ]:
Mdx = np.arange(M)
dz = eit*dt
for j in range(0, m+1):
    coff = j*M
    qj = -qv0(j) if j > 0 else 1.
    L[0,coff+Mdx] = (qj*rz[:,j]*dz).reshape(1, M)
    
zj = zb.T.reshape(1, (m+1)*M)
for k in range(1, N):
    L[k,:] = L[k-1,:]*zj

Following block-rows are for $p\in\{1,...,m\}$ and involve multiples of $q_p/(z_j - \delta_p)$.

In [ ]:
for p in range(1, m+1):
    roff = p*(N+1) - 1 # = N + (p-1)*(N+1)
    etap = qv[p-1]/(zj - dv[p-1])
    L[roff,:] = L[0,:]*etap
    for k in range(1, N+1):
        L[roff+k,:] = L[roff+k-1,:]*etap

Want matrix-vector multiplication to do the integration. But this isn't working.

rhs = L.dot(np.tile((eit*dt).reshape(M, 1), (m+1, 1)))

So sum instead?

In [ ]:
rhs = L.sum(axis=1).reshape(N + m*(N+1), 1)

Compare some things with reference.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
idx = np.arange(5)
abserr = np.abs(refRHS - rhs)
plt.semilogy(abserr, '.-')
plt.show()

================================
* Previous Compare for $p=0$

zkj = np.ones(D, dtype=complex)
for _ in range(1, -k):
    zkj *= zj

Ip0 = -qj*pi2/D*np.sum(rj(zj)*zkj*eti)
print(Ip0)

zjt = lambda t: dj + qj*np.exp(1j*t)
f = lambda t: rj(zjt(t))*zjt(t)**(-k-1)*np.exp(1j*t)
r1 = integrate.quad(lambda t: f(t).real, 0., pi2)
r2 = integrate.quad(lambda t: f(t).imag, 0., pi2)
Cp0 = -qj*(r1[0] + 1j*r2[0])
print(Cp0)

* Previous Compare for $p > 0$

p = 2
dp = dv[p-1]
qp = dv[p-1]

zjp = (zj - dp)/qp
zkj = 1./zjp
for _ in range(1, -k):
    zkj = zkj/zjp

Ip2 = -qj*pi2/D*np.sum(rj(zj)*zkj*eti)
print(Ip2)

f2 = lambda t: rj(zjt(t))*((zjt(t) - dp)/qp)**k*np.exp(1j*t)
r1 = integrate.quad(lambda t: f2(t).real, 0., pi2)
r2 = integrate.quad(lambda t: f2(t).imag, 0., pi2)
Cp2 = -qj*(r1[0] + 1j*r2[0])
print(Cp2)